In [4]:
import osmnx as ox
import networkx as nx
import random
from shapely.geometry import Point
import math
import numpy as np # added for potential future numerical operations

# define hospital class
class Hospital:
    def __init__(self, location, capacity, latlong):
        self.location = location  # node id in the graph
        self.capacity = capacity
        self.latlong = latlong # latitude and longitude coordinates
        self.beginning = capacity # store initial capacity for resetting

# define victim class
class Victim:
    def __init__(self, location, rating, latlong):
        self.location = location  # node id in the graph
        self.rating = rating      # 'r' (red/critical), 'g' (green), 'u' (unknown)
        self.visited = False      # flag to track if visited by a vehicle
        self.latlong = latlong    # latitude and longitude coordinates

# define vehicle class
class Vehicle:
    def __init__(self, vehicle_type, max_risk, location, capacity, index, latlong):
        self.type = vehicle_type     # e.g., 'ambulance', 'helicopter', 'rescue_truck'
        self.max_risk = max_risk     # maximum risk level the vehicle can tolerate on a path
        self.location = location     # current node id in the graph
        self.capacity = capacity     # max number of victims vehicle can carry
        self.current_amt = 0         # current number of victims in the vehicle
        self.route = []              # planned sequence of nodes for the vehicle's route
        self.risk_scores = []        # stores risk scores for segments of the route
        self.index = index           # unique identifier for the vehicle
        self.latlong = latlong       # latitude and longitude coordinates
        self.assigned_victims = []   # list of victim objects assigned to this vehicle

# helper function to get nearest node
def get_nearest_node(graph, latlong):
    """finds the closest node in the osmnx graph for given latitude and longitude."""
    point = Point(latlong[1], latlong[0]) # shapely point expects (longitude, latitude)
    return ox.nearest_nodes(graph, point.x, point.y)

# vehicle risk calculation function
def vehicle_risk(u, v, data, vehicle_type, vehicle_max_risk=5, alpha=0.5, beta=0.5):
    """calculates a combined score for traversing an edge based on time and risk.
       returns infinity if the edge's danger exceeds vehicle_max_risk.
    """
    # default edge speed for general network if 'maxspeed' is not present
    # osmnx automatically imputes missing speeds for 'drive' network_type if speed column not available.
    # we'll rely on the 'travel_time' attribute which osmnx adds if it's not present.
    travel_time = data.get('travel_time', 0) # assumes osmnx has calculated this

    # get danger score from edge data, default to 0 if not present
    temp_danger = data.get('danger', 0)

    # check if the edge is traversable based on vehicle's max_risk
    if temp_danger > vehicle_max_risk:
        return float('inf'), 0, 0  # not traversable for this vehicle type

    # calculate combined weight (score) for the edge
    # the formula can be adjusted based on real-world requirements
    combined_weight = alpha * travel_time + beta * temp_danger

    return combined_weight, travel_time, temp_danger

# graph initialization and data generation
# define the bounding box for the operational area
# using (west, south, east, north) for bbox parameter as per osmnx v2 migration guide
bbox_coords = (-95.6, 29.5, -95.2, 29.8) # roughly sugar land, tx area

# get a graph of the road network within the bounding box
print("downloading graph...")
# g = ox.graph_from_bbox(north, south, east, west, network_type='drive') # deprecated
g = ox.graph_from_bbox(bbox_coords[3], bbox_coords[1], bbox_coords[2], bbox_coords[0], network_type='drive') # corrected for v2.0.0
print("graph downloaded.")

# impute missing travel times (speeds) if not already present
g = ox.add_edge_speeds(g)
g = ox.add_edge_travel_times(g)

# add random danger scores to edges
for u, v, k, data in g.edges(keys=True, data=True):
    data['danger'] = random.randint(0, 10) # danger score from 0 to 10

all_nodes = list(g.nodes())

# generate hospitals
num_hospitals = 3
hospitals = []
print(f"generating {num_hospitals} hospitals...")
for i in range(num_hospitals):
    # get random coordinates within the bounding box
    rand_lat = random.uniform(bbox_coords[1], bbox_coords[3])
    rand_lon = random.uniform(bbox_coords[0], bbox_coords[2])
    latlong = (rand_lat, rand_lon)
    node = get_nearest_node(g, latlong)
    capacity = random.randint(5, 15) # random capacity between 5 and 15
    hospitals.append(Hospital(node, capacity, latlong))
    print(f"  hospital {i+1}: node {node}, capacity {capacity}, latlong {latlong}")

# generate victims
num_victims = 15
victims = []
victim_ratings = ['r'] * 3 + ['g'] * 7 + ['u'] * 5 # 3 red, 7 green, 5 unknown
random.shuffle(victim_ratings)

print(f"generating {num_victims} victims...")
for i in range(num_victims):
    rand_lat = random.uniform(bbox_coords[1], bbox_coords[3])
    rand_lon = random.uniform(bbox_coords[0], bbox_coords[2])
    latlong = (rand_lat, rand_lon)
    node = get_nearest_node(g, latlong)
    rating = victim_ratings[i]
    victims.append(Victim(node, rating, latlong))
    print(f"  victim {i+1}: node {node}, rating {rating}, latlong {latlong}")

# generate vehicles
num_vehicles = 4
vehicles = []
vehicle_types = ['ambulance', 'helicopter', 'rescue_truck'] # example types
type_props = {'ambulance': {'max_risk': 3, 'capacity': 2, 'speed_factor': 1.0}, # speed factor applied to travel time
              'helicopter': {'max_risk': 10, 'capacity': 1, 'speed_factor': 0.5}, # helicopters ignore roads, faster
              'rescue_truck': {'max_risk': 7, 'capacity': 3, 'speed_factor': 0.8}}

print(f"generating {num_vehicles} vehicles...")
for i in range(num_vehicles):
    v_type = random.choice(vehicle_types)
    props = type_props[v_type]
    # start all vehicles at a random hospital location
    start_hospital = random.choice(hospitals)
    vehicles.append(Vehicle(v_type, props['max_risk'], start_hospital.location, 
                            props['capacity'], i, start_hospital.latlong))
    print(f"  vehicle {i+1}: type {v_type}, max risk {props['max_risk']}, capacity {props['capacity']}, start node {start_hospital.location}")

print("\n--- initialization complete ---\n")

downloading graph...


/var/folders/j0/y6d4361s4c3gs1wcm65ym00w0000gn/T/ipykernel_85810/2234082844.py:75: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  g = ox.graph_from_bbox(bbox_coords[3], bbox_coords[1], bbox_coords[2], bbox_coords[0], network_type='drive') # corrected for v2.0.0
/var/folders/j0/y6d4361s4c3gs1wcm65ym00w0000gn/T/ipykernel_85810/2234082844.py:75: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  g = ox.graph_from_bbox(bbox_coords[3], bbox_coords[1], bbox_coords[2], bbox_coords[0], network_type='drive') # corrected for v2.0.0


graph downloaded.
generating 3 hospitals...
  hospital 1: node 2206010479, capacity 15, latlong (29.62978093396286, -95.39548268673163)
  hospital 2: node 152372026, capacity 6, latlong (29.782926525726563, -95.29960448822804)
  hospital 3: node 2057693500, capacity 5, latlong (29.693152051582423, -95.52325187661556)
generating 15 victims...
  victim 1: node 151458160, rating g, latlong (29.673287485486405, -95.2509915832608)
  victim 2: node 3807491884, rating u, latlong (29.53725186200973, -95.56820087690588)
  victim 3: node 3083161036, rating g, latlong (29.596216622341448, -95.2558242116263)
  victim 4: node 153182037, rating g, latlong (29.68027749215491, -95.297573481523)
  victim 5: node 222662101, rating g, latlong (29.592944982605175, -95.33845638379415)
  victim 6: node 152858611, rating u, latlong (29.74145913005381, -95.45165626890005)
  victim 7: node 222657119, rating r, latlong (29.531922474689217, -95.43163113351673)
  victim 8: node 2121574308, rating r, latlong (29.7

In [5]:
vehicle_matrices = {}

nodes_of_interest = set([v.location for v in victims] + [h.location for h in hospitals] + [veh.location for veh in vehicles])
nodes_of_interest = list(nodes_of_interest)

print("calculating shortest path matrices for each vehicle type...")
for vehicle_type in type_props.keys():
    vehicle_info = type_props[vehicle_type]
    max_risk_for_type = vehicle_info['max_risk']
    speed_factor_for_type = vehicle_info['speed_factor']

    type_matrix = {'score': {}, 'time': {}, 'risk': {}, 'path': {}}

    for source_node in nodes_of_interest:
        type_matrix['score'][source_node] = {}
        type_matrix['time'][source_node] = {}
        type_matrix['risk'][source_node] = {}
        type_matrix['path'][source_node] = {}

        for target_node in nodes_of_interest:
            if source_node == target_node:
                type_matrix['score'][source_node][target_node] = 0
                type_matrix['time'][source_node][target_node] = 0
                type_matrix['risk'][source_node][target_node] = 0
                type_matrix['path'][source_node][target_node] = [source_node]
                continue

            # use dijkstra's algorithm to find the shortest path
            # the 'weight' parameter here refers to the combined score from vehicle_risk
            try:
                path = nx.shortest_path(g, source=source_node, target=target_node, 
                                         weight=lambda u, v, data: vehicle_risk(u, v, data, vehicle_type, max_risk_for_type)[0])
                
                total_time = 0
                total_risk = 0
                for i in range(len(path) - 1):
                    u, v = path[i], path[i+1]
                    # find the specific edge (handle multi-edges)
                    edge_data = g.get_edge_data(u, v)
                    # take the first edge if multiple exist, or iterate if specific logic needed
                    edge_key = next(iter(edge_data)) # get the key of the first edge
                    data = edge_data[edge_key]
                    
                    # recalculate based on the edge data
                    _, time, risk = vehicle_risk(u, v, data, vehicle_type, max_risk_for_type)
                    total_time += time
                    total_risk += risk
                
                # apply speed factor for vehicle type
                total_time *= speed_factor_for_type
                
                # calculate the final score for the path
                # adjust weights as needed (e.g., risk is more penalizing)
                path_score = (total_time / 100) + (0.2 * total_risk)

                type_matrix['score'][source_node][target_node] = path_score
                type_matrix['time'][source_node][target_node] = total_time
                type_matrix['risk'][source_node][target_node] = total_risk
                type_matrix['path'][source_node][target_node] = path
            except nx.networkxnopath:
                # no path exists or path involves edges exceeding max_risk
                type_matrix['score'][source_node][target_node] = float('inf')
                type_matrix['time'][source_node][target_node] = float('inf')
                type_matrix['risk'][source_node][target_node] = float('inf')
                type_matrix['path'][source_node][target_node] = []
            except Exception as e:
                print(f"error calculating path for {vehicle_type} from {source_node} to {target_node}: {e}")
                type_matrix['score'][source_node][target_node] = float('inf')
                type_matrix['time'][source_node][target_node] = float('inf')
                type_matrix['risk'][source_node][target_node] = float('inf')
                type_matrix['path'][source_node][target_node] = []

    vehicle_matrices[vehicle_type] = type_matrix
    print(f"  matrix for {vehicle_type} complete.")

print("\n--- path matrix calculation complete ---\n")

calculating shortest path matrices for each vehicle type...
  matrix for ambulance complete.
  matrix for helicopter complete.
  matrix for rescue_truck complete.

--- path matrix calculation complete ---



In [6]:
def calculate_total_score(current_vehicles, victims_list, hospitals_list, vehicle_matrices):
    """calculates the total score for a given state (vehicle assignments and routes).
    lower score is better.
    """
    total_score = 0
    visited_victims_count = 0
    # reset capacities for calculation
    for h in hospitals_list:
        h.capacity = h.beginning
    for v in current_vehicles:
        v.current_amt = 0
        v.route = [] # clear route before recalculating
        v.risk_scores = []
    for vic in victims_list:
        vic.visited = False

    # assign victims to vehicles and build routes based on assigned_victims
    for vehicle in current_vehicles:
        current_loc = vehicle.location # vehicle's starting location
        vehicle_type = vehicle.type
        
        # sort assigned victims by rating to prioritize 'r' first (red, then unknown, then green)
        # if ratings are equal, order doesn't strictly matter for initial assignment, but path order might.
        # for now, we'll process them in the order they were assigned during neighbor generation, which should be fine.
        # the scoring mechanism heavily penalizes unvisited 'r' victims.

        vehicle_path_score = 0
        path_segments = [] # to store (source, target) for route construction

        # first, ensure red victims are processed as highest priority
        red_victims_assigned = [v for v in vehicle.assigned_victims if v.rating == 'r']
        other_victims_assigned = [v for v in vehicle.assigned_victims if v.rating != 'r']
        
        # prioritize red victims first, then others
        sorted_assigned_victims = red_victims_assigned + other_victims_assigned
        
        # create a copy to iterate, allowing modification of original list if needed (though not done here)
        assigned_victims_copy = list(sorted_assigned_victims)

        for victim in assigned_victims_copy:
            if vehicle.current_amt < vehicle.capacity:
                try:
                    path_score = vehicle_matrices[vehicle_type]['score'][current_loc][victim.location]
                    travel_time = vehicle_matrices[vehicle_type]['time'][current_loc][victim.location]
                    risk_taken = vehicle_matrices[vehicle_type]['risk'][current_loc][victim.location]
                    path_nodes = vehicle_matrices[vehicle_type]['path'][current_loc][victim.location]
                    
                    if math.isinf(path_score): # check if path is inaccessible
                        # if a path is inaccessible, this assignment is invalid, penalize heavily
                        total_score += float('inf')
                        continue # skip to next vehicle or victim

                    vehicle_path_score += path_score
                    vehicle.current_amt += 1
                    victim.visited = True
                    visited_victims_count += 1
                    
                    # add cost for victim rating based on travel time
                    if victim.rating == 'r':
                        vehicle_path_score += 0.5 * travel_time # higher penalty for critical victims
                    elif victim.rating == 'u':
                        vehicle_path_score += 0.2 * travel_time # medium penalty for unknown victims
                    
                    # extend vehicle's route and risk scores
                    if not vehicle.route: # if this is the first segment, include current_loc
                        vehicle.route.extend(path_nodes)
                    else:
                        vehicle.route.extend(path_nodes[1:]) # append from second node to avoid duplication
                    vehicle.risk_scores.append(risk_taken)
                    
                    current_loc = victim.location # update current location for next segment
                except keyerror:
                    # this can happen if a node is isolated or an assignment is invalid
                    total_score += float('inf')
                    continue
            else: # vehicle is full, must go to hospital
                # find nearest hospital with capacity
                best_hospital = None
                min_hospital_score = float('inf')
                for h in hospitals_list:
                    if h.capacity > 0:
                        try:
                            hospital_score = vehicle_matrices[vehicle_type]['score'][current_loc][h.location]
                            if hospital_score < min_hospital_score:
                                min_hospital_score = hospital_score
                                best_hospital = h
                        except keyerror:
                            pass # no path to this hospital
                
                if best_hospital:
                    hospital_path_score = vehicle_matrices[vehicle_type]['score'][current_loc][best_hospital.location]
                    hospital_path_nodes = vehicle_matrices[vehicle_type]['path'][current_loc][best_hospital.location]
                    hospital_risk_taken = vehicle_matrices[vehicle_type]['risk'][current_loc][best_hospital.location]

                    vehicle_path_score += hospital_path_score
                    best_hospital.capacity -= vehicle.current_amt # drop off all victims
                    vehicle.current_amt = 0 # vehicle is now empty
                    
                    if not vehicle.route:
                        vehicle.route.extend(hospital_path_nodes)
                    else:
                        vehicle.route.extend(hospital_path_nodes[1:])
                    vehicle.risk_scores.append(hospital_risk_taken)
                    current_loc = best_hospital.location # vehicle is now at hospital
    
    # after visiting victims and potentially going to a hospital, 
    # if the vehicle has victims, it must drop them off.
    # if the vehicle has visited victims and is not at a hospital, route to nearest hospital.
    if vehicle.current_amt > 0: # vehicle still has victims
        best_hospital = None
        min_hospital_score = float('inf')
        for h in hospitals_list:
            if h.capacity > 0: # ensure hospital has capacity
                try:
                    hospital_score = vehicle_matrices[vehicle_type]['score'][current_loc][h.location]
                    if hospital_score < min_hospital_score:
                        min_hospital_score = hospital_score
                        best_hospital = h
                except keyerror:
                    pass
        
        if best_hospital:
            hospital_path_score = vehicle_matrices[vehicle_type]['score'][current_loc][best_hospital.location]
            hospital_path_nodes = vehicle_matrices[vehicle_type]['path'][current_loc][best_hospital.location]
            hospital_risk_taken = vehicle_matrices[vehicle_type]['risk'][current_loc][best_hospital.location]

            vehicle_path_score += hospital_path_score
            best_hospital.capacity -= vehicle.current_amt
            vehicle.current_amt = 0
            
            if not vehicle.route:
                vehicle.route.extend(hospital_path_nodes)
            else:
                vehicle.route.extend(hospital_path_nodes[1:])
            vehicle.risk_scores.append(hospital_risk_taken)
            current_loc = best_hospital.location
        else:
            # no hospital with capacity found, penalize heavily
            vehicle_path_score += float('inf')

    # add vehicle's total path score to the overall total
    total_score += vehicle_path_score

    # penalize unvisited 'red' victims most heavily
    for vic in victims_list:
        if vic.rating == 'r' and not vic.visited:
            total_score += 5000 # high penalty for unvisited critical victims

    return total_score

In [7]:
def get_neighbor(current_vehicles, victims_list, hospitals_list, vehicle_matrices):
    """generates a new neighboring solution by slightly modifying the current one.
       this involves reassigning victims or swapping assignments between vehicles.
    """
    # create deep copies to avoid modifying the original solution directly
    new_vehicles = []
    for v in current_vehicles:
        new_vehicle = Vehicle(v.type, v.max_risk, v.location, v.capacity, v.index, v.latlong)
        # deep copy assigned_victims - ensure it's a list of new victim objects
        new_vehicle.assigned_victims = []
        for old_victim in v.assigned_victims:
            new_victim_copy = Victim(old_victim.location, old_victim.rating, old_victim.latlong)
            new_vehicle.assigned_victims.append(new_victim_copy)
        new_vehicles.append(new_vehicle)

    new_victims = []
    victim_map = {vic.location: vic for vic in victims_list} # map victim node to victim object
    for vic in victims_list:
        new_victims.append(Victim(vic.location, vic.rating, vic.latlong))

    new_hospitals = []
    for h in hospitals_list:
        new_hospitals.append(Hospital(h.location, h.capacity, h.latlong))

    # --- perturbation logic ---
    action = random.choice(['swap_victims_between_vehicles', 'move_victim_to_vehicle', 'swap_vehicle_assignments'])

    if action == 'swap_victims_between_vehicles' and len(new_vehicles) >= 2:
        # attempt to swap victims between two randomly chosen vehicles
        v1, v2 = random.sample(new_vehicles, 2)
        if v1.assigned_victims and v2.assigned_victims:
            vic1_idx = random.randrange(len(v1.assigned_victims))
            vic2_idx = random.randrange(len(v2.assigned_victims))
            
            # swap the victim objects directly (they are copies)
            temp_vic = v1.assigned_victims[vic1_idx]
            v1.assigned_victims[vic1_idx] = v2.assigned_victims[vic2_idx]
            v2.assigned_victims[vic2_idx] = temp_vic

    elif action == 'move_victim_to_vehicle' and new_victims and new_vehicles:
        # attempt to move a random unassigned victim to a random vehicle, or reassign an assigned one
        target_victim = random.choice(new_victims)
        target_vehicle = random.choice(new_vehicles)
        
        # remove victim from its current assignment if it's already assigned
        for v in new_vehicles:
            if target_victim in v.assigned_victims:
                v.assigned_victims.remove(target_victim)
                break
        
        # assign victim to the new vehicle
        target_vehicle.assigned_victims.append(target_victim)

    elif action == 'swap_vehicle_assignments' and new_victims and len(new_vehicles) >= 2:
        # pick two random victims and try to assign them to two random vehicles (possibly same)
        vic_to_assign1 = random.choice(new_victims)
        vic_to_assign2 = random.choice(new_victims)
        veh1, veh2 = random.sample(new_vehicles, 2)

        # remove victims from their current assignments
        for v in new_vehicles:
            if vic_to_assign1 in v.assigned_victims:
                v.assigned_victims.remove(vic_to_assign1)
            if vic_to_assign2 in v.assigned_victims:
                v.assigned_victims.remove(vic_to_assign2)
        
        # assign them to the new vehicles
        veh1.assigned_victims.append(vic_to_assign1)
        veh2.assigned_victims.append(vic_to_assign2)

    # important: ensure 'r' (red) victims are always assigned if possible.
    # this logic can be more sophisticated (e.g., ensure they are assigned first in a route).
    # for simplicity here, we'll just check if any 'r' victims are unassigned and assign them to a random available vehicle.
    assigned_victim_locations = set()
    for v in new_vehicles:
        assigned_victim_locations.update([vic.location for vic in v.assigned_victims])
    
    unassigned_red_victims = [vic for vic in new_victims if vic.rating == 'r' and vic.location not in assigned_victim_locations]
    
    for red_vic in unassigned_red_victims:
        if new_vehicles:
            # assign to a random vehicle, preferably one that can handle it
            # a more intelligent assignment might pick the closest vehicle or least loaded
            chosen_vehicle = random.choice(new_vehicles)
            chosen_vehicle.assigned_victims.append(red_vic)
            assigned_victim_locations.add(red_vic.location)

    # --- route re-construction and validation (simplified) ---
    # the actual paths and scores will be recalculated in calculate_total_score
    # here, we mainly update the assigned_victims list for each vehicle.
    # sort assigned victims within each vehicle: prioritize 'r' victims, then others.
    for vehicle in new_vehicles:
        red_vics = sorted([v for v in vehicle.assigned_victims if v.rating == 'r'], 
                          key=lambda x: vehicle_matrices[vehicle.type]['score'].get(vehicle.location, {}).get(x.location, float('inf')))
        other_vics = sorted([v for v in vehicle.assigned_victims if v.rating != 'r'], 
                            key=lambda x: vehicle_matrices[vehicle.type]['score'].get(vehicle.location, {}).get(x.location, float('inf')))
        vehicle.assigned_victims = red_vics + other_vics # red victims first

    return new_vehicles, new_victims, new_hospitals # return modified copies

# cell 11: simulated annealing algorithm
def simulated_annealing(vehicles_initial, victims_list, hospitals_list, vehicle_matrices, 
        initial_temperature, cooling_rate, iterations):
    """implements the simulated annealing metaheuristic for optimization."""
    
    # make initial deep copies for the algorithm to work with
    current_vehicles = []
    for v in vehicles_initial:
        new_vehicle = Vehicle(v.type, v.max_risk, v.location, v.capacity, v.index, v.latlong)
        # no assigned victims yet for initial state, will be assigned by get_neighbor if it takes all victims
        current_vehicles.append(new_vehicle)

    current_victims = []
    for vic in victims_list:
        current_victims.append(Victim(vic.location, vic.rating, vic.latlong))

    current_hospitals = []
    for h in hospitals_list:
        current_hospitals.append(Hospital(h.location, h.capacity, h.latlong))

    # initialize 'current_solution' to a random valid assignment first
    # this is a crucial step for sa - ensure the initial solution is reasonably valid.
    # simple initial assignment: distribute victims round-robin or based on proximity
    # for now, we'll rely on `get_neighbor` to generate a sensible first state from a 'blank' initial state.

    # start with a random initial assignment for the 'current_solution'
    temp_current_vehicles, temp_current_victims, temp_current_hospitals = \
        get_neighbor(current_vehicles, current_victims, current_hospitals, vehicle_matrices)
    
    # distribute all victims initially to get a starting point
    unassigned_vics = list(temp_current_victims) # all victims are initially unassigned in this context
    random.shuffle(unassigned_vics)
    
    for i, vic in enumerate(unassigned_vics):
        if temp_current_vehicles:
            # assign to vehicles in a round-robin fashion
            assigned_vehicle_idx = i % len(temp_current_vehicles)
            temp_current_vehicles[assigned_vehicle_idx].assigned_victims.append(vic)

    current_vehicles = temp_current_vehicles
    current_victims = temp_current_victims
    current_hospitals = temp_current_hospitals

    current_score = calculate_total_score(current_vehicles, current_victims, current_hospitals, vehicle_matrices)
    best_score = current_score
    best_vehicles = current_vehicles
    best_victims = current_victims # store victims state associated with best vehicles
    
    temperature = initial_temperature

    print("starting simulated annealing...")
    print(f"initial score: {current_score:.2f}")

    for i in range(iterations):
        # generate a neighbor solution
        neighbor_vehicles, neighbor_victims, neighbor_hospitals = \
            get_neighbor(current_vehicles, victims_list, hospitals_list, vehicle_matrices)
        
        neighbor_score = calculate_total_score(neighbor_vehicles, neighbor_victims, neighbor_hospitals, vehicle_matrices)

        # calculate energy difference
        delta_e = neighbor_score - current_score
        
        # acceptance probability
        if delta_e < 0 or (temperature > 0 and random.uniform(0, 1) < math.exp(-delta_e / temperature)):
            current_score = neighbor_score
            current_vehicles = neighbor_vehicles
            current_victims = neighbor_victims
            current_hospitals = neighbor_hospitals
            
            if current_score < best_score:
                best_score = current_score
                best_vehicles = current_vehicles # store the vehicles for the best solution
                best_victims = current_victims # store victims state for consistency

        # cool down the temperature
        temperature *= cooling_rate
        
        if i % (iterations // 10) == 0: # print progress every 10% of iterations
            print(f"iteration {i}/{iterations}, temp: {temperature:.2f}, current score: {current_score:.2f}, best score: {best_score:.2f}")
        
        if temperature < 1e-5: # stop if temperature gets too low
            print(f"temperature too low, stopping at iteration {i}")
            break

    print("\nsimulated annealing finished.")
    print(f"final best score: {best_score:.2f}")
    
    # recalculate the route and visited status for the best solution before returning
    # this ensures the returned 'best_vehicles' has its 'route', 'risk_scores', 'current_amt' 
    # and associated 'best_victims' have their 'visited' status correctly set.
    final_score = calculate_total_score(best_vehicles, best_victims, hospitals_list, vehicle_matrices)

    return best_score, best_vehicles

# cell 12: execute simulated annealing
# ensure 'vehicles', 'victims', 'hospitals', and 'vehicle_matrices' are defined from previous cells.

initial_temp = 100
cooling_rate = 0.95
num_iterations = 10000

final_best_score, final_best_vehicles = simulated_annealing(
    vehicles,
    victims,
    hospitals,
    vehicle_matrices,
    initial_temp,
    cooling_rate,
    num_iterations
)

print("\n--- optimized solution details ---")
print(f"total score: {final_best_score:.2f}")
for i, vehicle in enumerate(final_best_vehicles):
    print(f"\nvehicle {vehicle.index} ({vehicle.type}):")
    if vehicle.assigned_victims:
        print(f"  assigned victims: {[f'victim at {v.location} ({v.rating})' for v in vehicle.assigned_victims]}")
        print(f"  route (nodes): {vehicle.route}")
        print(f"  risk scores (per segment): {vehicle.risk_scores}")
    else:
        print("  no victims assigned.")

starting simulated annealing...
initial score: 507.78
iteration 0/10000, temp: 95.00, current score: 507.78, best score: 507.78
temperature too low, stopping at iteration 314

simulated annealing finished.
final best score: 507.78

--- optimized solution details ---
total score: 507.78

vehicle 0 (rescue_truck):
  assigned victims: ['victim at 2121574308 (r)', 'victim at 7461429089 (g)', 'victim at 222662101 (g)', 'victim at 152933738 (u)', 'victim at 152364343 (g)']
  route (nodes): [2206010479, 2390135386, 2390135384, 2520097422, 2206911024, 2390135380, 1139583175, 773614386, 153472745, 153472747, 9765537435, 8631844047, 8632191427, 8189056401, 7055938099, 152505659, 152505643, 152178750, 347264732, 347264850, 151674670, 151590146, 347274225, 152002965, 347274933, 152009496, 151812062, 151833541, 152621990, 347437887, 347437947, 330721077, 151903394, 1046366919, 153279226, 151586406, 347438885, 151615032, 590324637, 1342896051, 151944002, 153279160, 243998665, 152511060, 330754700, 1